In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Wczytanie danych

In [ ]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))
# train_additional = pd.read_csv(os.path.join('data', 'train_layer.csv'))

# train_additional.info()

In [ ]:
train.info()

### Rozbicie daty na składowe

In [ ]:
def change_date(dataframe, sin_cos=True):
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['dayofweek'] = dataframe['date'].dt.dayofweek.astype(np.int64)
    dataframe['week'] = dataframe['date'].dt.isocalendar().week.astype(np.int64)
    dataframe['year'] = dataframe['date'].dt.year.astype(np.int64)
    if 'month' not in dataframe.columns:
        dataframe['month'] = dataframe['date'].dt.month.astype(np.int64)
    # if sin_cos:
    #     dataframe['sin_dayofweek'] = np.sin(2 * np.pi * dataframe['dayofweek'] / 7)
    #     dataframe['cos_dayofweek'] = np.cos(2 * np.pi * dataframe['dayofweek'] / 7)
    #     dataframe['sin_week'] = np.sin(2 * np.pi * dataframe['week'] / 52)
    #     dataframe['cos_week'] = np.cos(2 * np.pi * dataframe['week'] / 52)
    #     dataframe['sin_month'] = np.sin(2 * np.pi * dataframe['month'] / 12)
    #     dataframe['cos_month'] = np.cos(2 * np.pi * dataframe['month'] / 12)
        
    return dataframe


train, test = change_date(train), change_date(test)
# train = train[(train['date'] >= test['date'].min()) & (train['date'] <= test['date'].max())]
# train.reset_index(drop=True, inplace=True)
# train_additional = change_date(train_additional)

### Wykres przedstawiający wartość pm2_5 w zarejestrowanych godzinach

In [ ]:
sns.barplot(data=train, x='hour', y='pm2_5')
plt.title('Jakość powietrza w poszczególnych godzinach')

### Wykres przedstawiający wartość pm2_5 w zależności od dnia tygodnia

In [ ]:
sns.barplot(data=train, x='dayofweek', y='pm2_5')
plt.title('Jakość powietrza w każdym dniu tygodnia')

### Wykres przedstawiający wartość pm2_5 w zależności od miesiąca

In [ ]:
sns.barplot(data=train, x='month', y='pm2_5')
plt.title('Jakość powietrza w każdym miesiącu')

### Korelacja wybranych kolumn z pm2_5

In [ ]:
sns.heatmap(train[['month', 'week', 'dayofweek', 'year', 'hour', 'site_latitude', 'site_longitude', 'cloud_surface_albedo', 'pm2_5']].corr(), annot=True, cmap='Greys')

### Wykres przedstawiający wartości pm2_5 w poszczególnych miastach

In [ ]:
plt.figure(figsize=(14, 8))
sns.lineplot(data=train, x='date', y='pm2_5', hue='city')
plt.title('Jakość powietrza w kolejnych dniach')

### Wartości pm2_5 dla Lagos z podziałem na site_id

In [ ]:
only_lagos = train[train['city'] == 'Lagos'][['site_id', 'pm2_5']]

only_lagos.plot(kind='hist', x='site_id')

### Kolumna ozone_o3_effective_temperature

In [ ]:
train['ozone_o3_effective_temperature'].describe()

In [ ]:
ozone_o3_effective_temperature = train.loc[train['ozone_o3_effective_temperature'].isna(), ['city', 'week', 'dayofweek', 'month', 'pm2_5']].copy(deep=True)

In [ ]:
ozone_o3_effective_temperature.groupby('city')['month'].value_counts()

In [ ]:
train.groupby(['city', 'month'])['ozone_o3_effective_temperature'].mean()

## <center>Czyszczenie danych</center>
### Usunięcie wszystkich kolumn od sensor

możliwa zależność z położeniem miasta

In [ ]:
train.drop(columns=[column for column in train.columns if 'sensor_zenith' in column or 'sensor_azimuth' in column], inplace=True)
test.drop(columns=[column for column in test.columns if 'sensor_zenith' in column or 'sensor_azimuth' in column], inplace=True)

### 1. Imputacja, usuwanie kolumn, oraz inne cuda

In [ ]:
from sklearn.impute import KNNImputer


def fill_train(column_name='dayofweek'):
    column_values = train[column_name].unique()
    for date_unit in column_values:
        for i, col in enumerate(starts_with):
            similar_columns = [col for col in train.columns if col.startswith(col)]
            df = train.loc[train[column_name] == date_unit, similar_columns].copy()
            if not df.empty:
                try:
                    train.loc[train[column_name] == date_unit, similar_columns] = imputers[i].fit_transform(df)
                except ValueError:
                    train.drop(index=df.index, inplace=True)
                    train.reset_index(drop=True, inplace=True)

def fill_test(column_name='dayofweek'):
    column_values = test[column_name].unique()
    for date_unit in column_values:
        for i, col in enumerate(starts_with):
            similar_columns = [col for col in test.columns if col.startswith(col)]
            df = test.loc[test[column_name] == date_unit, similar_columns].copy()
            if not df.empty:
                test.loc[test[column_name] == date_unit, similar_columns] = imputers[i].transform(df)

def drop_high_nans():
    columns_nans = []
    for i, el in enumerate(train.columns):
        if train[el].isna().sum() / len(train) >= 0.9:
            columns_nans.append(el)
    return columns_nans

def drop_high_correlated_columns():
    matrix = train.corr(numeric_only=True).abs()
    upper_t = matrix.where(np.triu(np.ones_like(matrix, dtype=np.bool_), k=1))
    return [col for col in upper_t.columns if any(upper_t[col] > 0.99)]

def drop_low_correlated_columns_to_pm2_5():
    corr = train.corr(numeric_only=True)['pm2_5'].to_frame()
    return corr[(corr['pm2_5'] < 0.01) & (corr['pm2_5'] > -0.01)].index.to_numpy()

def merge_azimuth_zenith(dataframe):
    zenith_columns = [zenith for zenith in dataframe.columns if 'zenith' in zenith]
    azimuth_columns = [azimuth for azimuth in dataframe.columns if 'azimuth' in azimuth]
    for i, zenith in enumerate(zenith_columns):
        splitted = zenith.split('_')
        dataframe[f'{splitted[0]}_{splitted[1]}_merged'] = dataframe[zenith] * dataframe[azimuth_columns[i]]
        dataframe.drop(zenith_columns[i], axis=1, inplace=True)
        dataframe.drop(azimuth_columns[i], axis=1, inplace=True)
    return dataframe


starts_with = train.columns.str.split('_', expand=True).levels[0].to_frame()
starts_with.drop(['month', 'pm2', 'id', 'city', 'date', 'country', 'site', 'hour', 'week', 'year', 'dayofweek'], inplace=True)
starts_with = starts_with[0].tolist()

imputers = [KNNImputer(n_neighbors=15, weights='distance') for _ in range(len(starts_with))]
high_nans = drop_high_nans()
train, test = train.drop(columns=high_nans, axis=1), test.drop(columns=high_nans, axis=1)
# fill_train(), fill_test()
# to_drop = drop_low_correlated_columns_to_pm2_5()
# to_drop = np.concatenate((to_drop, drop_high_correlated_columns()), axis=0)
# train, test = train.drop(columns=to_drop, axis=1), test.drop(columns=to_drop, axis=1)
# train, test = merge_azimuth_zenith(train), merge_azimuth_zenith(test)

train.info()

### Imputacja danych przy użyciu dat

In [ ]:
import calendar


def draw_calendar(dates_to_highlight, year):
    plt.rcParams.update({'font.size': 12})
    cal = calendar.Calendar(firstweekday=0)
    fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(15, 10))
    for m in range(12):
        x, y = divmod(m, 3)
        month_days = cal.monthdayscalendar(year, m + 1)
        
        ax[x, y].set_xlim(0, 7)
        ax[x, y].set_ylim(0, len(month_days))
        
        ax[x, y].axis('off')
        
        days_of_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        for i, day in enumerate(days_of_week):
            ax[x, y].text(i + 0.5, len(month_days) - 0.5, day, ha='center', va='center', weight='bold')
        
        for r, week in enumerate(month_days):
            for c, day in enumerate(week):
                if day != 0:
                    date_unit = pd.Timestamp(year, m + 1, day)
                    highlight = date_unit in dates_to_highlight
                    ax[x, y].text(c + 0.5, len(month_days) - r - 1.5, day, ha='center', va='center', 
                            bbox=dict(facecolor='red' if highlight else 'white', edgecolor='black'))
        ax[x, y].set_title(f'Calendar {calendar.month_name[m + 1]} {year}')
    plt.tight_layout()
    plt.show()


columns_worth_filling = [col for col in train.columns if train[col].isna().sum() != 0 and 0.5 < (train[col].notna().sum() / len(train)) < 0.9]
draw_calendar(set(train.loc[train['carbonmonoxide_co_column_number_density'].isna(), 'date']), year=2023)

### ACF, PACF

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


def plot_correlations():
    split_worth_filling = [[col for col in columns_worth_filling if start_with in col] for start_with in starts_with]
    split_worth_filling = [lst for lst in split_worth_filling if lst]
    for to_plot in split_worth_filling:
        fig, ax = plt.subplots(nrows=len(to_plot), ncols=2, figsize=(15, 30))
        for i, col in enumerate(to_plot):
            plot_acf(
                x=train[col],
                ax=ax[i, 0],
                missing='drop',
                title=f'ACF of {col}',
            )
            plot_pacf(
                x=train[col].dropna(),
                ax=ax[i, 1],
                title=f'PACF of {col}',
            )
        plt.tight_layout()
        plt.show()
        
plot_correlations()

### Wybór kolumn do wypełnienia danymi

In [ ]:
choosen_ones = [column for column in train.columns if '_solar_azimuth_angle' in column] + ['cloud_surface_albedo']

### Przed imputacją

In [ ]:
def plot_choosen_ones():
    fig, ax = plt.subplots(nrows=len(choosen_ones) // 4, ncols=4, figsize=(45, 30))
    for i, col in enumerate(choosen_ones):
        x, y = divmod(i, 4)
        sns.lineplot(data=train.loc[:, ['date', 'city', col]], x='date', y=col, hue='city', ax=ax[x, y])
        ax[x, y].set_title(col)
    plt.tight_layout()
    plt.show()
    
    
# plot_choosen_ones()

### Sprawdzenie czy szereg jest stacjonarny

In [ ]:
from statsmodels.tsa.stattools import adfuller


if_stationary = [{column: adfuller(train[column].dropna())} for column in choosen_ones]

### Imputacja przy wykorzystaniu ARIMA

In [ ]:
# from statsmodels.tsa.arima.model import ARIMA
# 
# 
# cities = train['city'].unique()
# for column in choosen_ones:
#     for city in cities:
#         data = train.loc[train['city'] == city, ['date', column]].groupby('date').mean()
#         data.index = pd.DatetimeIndex(data.index).to_period('D')
#         impute_model = ARIMA(
#             endog=data,
#             order=(1, 0, 1),
#             missing='none'
#         ).fit()
#         
#         na_dates = data.loc[data[column].isna(), :].index
#         for date in na_dates:
#             data.at[pd.to_datetime(pd.to_datetime(str(date))), column] = impute_model.predict(start=date, end=date).values[0]
#             
#         for index, value in data.iterrows():
#             train.loc[(train['date'] == pd.to_datetime(str(index))) & (train['city'] == city), column] = value.iloc[0]

### Po imputacji

In [ ]:
# plot_choosen_ones()

### Połączenie danych z innych konkursów

In [ ]:
train.info()

### Wykresy pudełkowe wskazujące wartości odstające

In [ ]:
from sympy import divisors


def plot_boxplots():
    for i, column_group in enumerate(starts_with):
        similar_columns = [col for col in train.columns if col.startswith(column_group)]
        if len(similar_columns) > 1:
            divs = divisors(len(similar_columns))
            if len(divs) % 2 == 0:
                rows, cols = divs[(len(divs) // 2) - 1], divs[len(divs) // 2]
            else:
                rows, cols = divs[len(divs) // 2], divs[len(divs) // 2]
            fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(40, 30), squeeze=False)
            fig.suptitle(column_group, fontsize=25)
            for j, col in enumerate(similar_columns):
                x_cord, y_cord = divmod(j, cols)
                train[col].plot(kind='box', ax=ax[x_cord, y_cord], fontsize=15)
            plt.show()


vertical_columns = [col for col in train.columns if 'number_density' in col]

# plot_boxplots()

### 2. Wskazanie kwantyli, od których są outliery

In [ ]:
from scipy.stats import zscore


detect_outliers = zscore(train['pm2_5'])

quantiles = pd.DataFrame(list(zip(np.linspace(0.97, 1, 21), [np.quantile(detect_outliers, el) for el in np.linspace(0.97, 1, 21)], [np.quantile(train['pm2_5'], el) for el in np.linspace(0.97, 1, 21)])), columns=['quantile', 'zscore', 'pm2_5'])
quantiles

### 3. Usunięcie wartości odstających

In [ ]:
def del_pm2_5_outliers():
    indexes_to_drop = []
    q1, q2 = np.quantile(detect_outliers, 0.01), np.quantile(detect_outliers, 0.99)
    for i, el in enumerate(detect_outliers):
        if el > q2:
            indexes_to_drop.append(i)
    train.drop(indexes_to_drop, inplace=True)
    train.reset_index(drop=True, inplace=True)
    
def cap_pm2_5_outliers():
    to_cap = np.quantile(detect_outliers, 0.99) * train['pm2_5'].std()
    train['pm2_5'] = train['pm2_5'].clip(upper=to_cap)


del_pm2_5_outliers()
# cap_pm2_5_outliers()

train.info()

### Usuwanie outlierów na podstawie rady marching_learning

In [ ]:
joined = pd.merge(only_lagos, detect_outliers, left_index=True, right_index=True)
outliers_from_lagos = joined[joined['pm2_5_y'] > 3.0]['site_id'].value_counts()
site_ids_to_del = outliers_from_lagos.keys()[:2]
outliers_from_lagos

In [ ]:
# indexes = train[train['site_id'].isin(site_ids_to_del)].index
# train.drop(index=indexes, inplace=True)
# train.reset_index(drop=True, inplace=True)

# train.info()

### Limit wartości dla kolumn uvaerosolindex_sensor_altitude
ustawienie zakresu z zbioru test

In [ ]:
# limit = train.loc[(test['uvaerosolindex_sensor_altitude'].min() < train['uvaerosolindex_sensor_altitude']) & (train['uvaerosolindex_sensor_altitude'] > test['uvaerosolindex_sensor_altitude'].max()), :].index
# train.drop(index=limit, inplace=True)
# train.reset_index(drop=True, inplace=True)

## <center>Selekcja cech</center>

In [ ]:
from sklearn.feature_selection import RFECV, RFE, SelectKBest, mutual_info_regression, f_regression
from sklearn.ensemble import RandomForestRegressor


def plot_feature_importance(sc, num_of_features):
    if isinstance(sc, RFECV) or isinstance(sc, RFE):
        scores = dict(zip(sc.feature_names_in_, sc.ranking_))
    else:
        scores = dict(zip(sc.feature_names_in_, sc.scores_))
    scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:num_of_features]
    scores_df = pd.DataFrame(scores, columns=['Feature', 'Score'])

    scores_df.plot(kind='bar', x='Feature', y='Score', figsize=(10, 6), rot=90, title='Oceny wybranych cech')
    plt.xlabel('Cecha')
    plt.ylabel('Ocena')


# selector = RFE(
#     estimator=RandomForestRegressor(
#         n_estimators=700, 
#         max_depth=7, 
#         random_state=4, 
#         n_jobs=-1, 
#         oob_score=True,
#         warm_start=True
#     ),
#     n_features_to_select=k,
# )
# k = 17
# selector = RFECV(
#     estimator=RandomForestRegressor(
#         n_estimators=400, 
#         max_depth=10, 
#         random_state=4, 
#         n_jobs=-1, 
#         oob_score=True, 
#         warm_start=True, 
#         ccp_alpha=1e-4
#     ),
#     min_features_to_select=k, 
#     cv=10, 
#     scoring='neg_root_mean_squared_error',
#     n_jobs=-1
# )
# selector.fit(train, y)
# train, test = selector.transform(train), selector.transform(test)
# 
# plot_feature_importance(selector, k)

## <center>Transformacja danych</center>
### 1. Standaryzacja danych

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler, TargetEncoder
from sklearn.compose import make_column_transformer


test_ids = test['id']
train.drop(columns=['id', 'city', 'date', 'country', 'site_id', 'site_latitude', 'site_longitude', 'hour'], inplace=True)
test.drop(columns=['id', 'city', 'date', 'country', 'site_id', 'site_latitude', 'site_longitude', 'hour'], inplace=True)

train_pm2_5 = train['pm2_5']
train.drop(columns=['pm2_5'], inplace=True)

#### Z podziałem na kategoryczne

In [ ]:
# categorical = ['month', 'dayofweek', 'week', 'year']
# train[categorical] = train[categorical].astype('category')
# test[categorical] = test[categorical].astype('category')
# 
# non_cat_columns = train.columns.difference(categorical)
# 
# non_cat_scaler = make_column_transformer((RobustScaler(), non_cat_columns))
# te = TargetEncoder(random_state=4)
# 
# train_cat, test_cat = te.fit_transform(train[categorical], train_pm2_5), te.transform(test[categorical])
# 
# train = pd.concat([pd.DataFrame(non_cat_scaler.fit_transform(train[non_cat_columns]), columns=non_cat_scaler.feature_names_in_), pd.DataFrame(train_cat, columns=te.feature_names_in_), train_pm2_5], axis=1)
# test = pd.concat([test_ids, pd.DataFrame(non_cat_scaler.transform(test[non_cat_columns]), columns=non_cat_scaler.feature_names_in_), pd.DataFrame(test_cat, columns=te.feature_names_in_)], axis=1)

# train = train.round(decimals=3)

#### Wszystkie kolumny

In [ ]:
scaler = StandardScaler()

train, test = scaler.fit_transform(train), scaler.transform(test)
train, test = pd.concat([pd.DataFrame(train, columns=scaler.feature_names_in_), train_pm2_5], axis=1), pd.concat([test_ids, pd.DataFrame(test, columns=scaler.feature_names_in_)], axis=1)

### Zapis przekształconych danych

In [ ]:
if 'transform' not in os.listdir(os.getcwd()):
    os.mkdir('transform')
train.to_csv(os.path.join('transform', 'train.csv'), index=False)
test.to_csv(os.path.join('transform', 'test.csv'), index=False)

In [ ]:
train.info()

In [ ]:
test.info()

### Po prostu xd

In [ ]:
xd = pd.read_csv(os.path.join('data', 'lightgbm.csv'), header=0)
xd['pm2_5'] *= 0.99
xd.to_csv(os.path.join('data', 'lightgbm_even_better.csv'), index=False)